In [1]:
import numpy as np
import pandas as pd
import os
import pickle
from pathlib import Path
import psutil
import re
import matplotlib.pyplot as plt

In [2]:
root_path = "C://Users//franc//Documents//Mapping_Recurrent_Inhibition_minimal_datasets_to_run_scripts//experimental_data//"
mu_properties_filename = "MU_properties_ALL.csv"

In [3]:
# HELPER FUNCTIONS

def extract_number(fname):
    """
    Finds a number in `fname` and returns it as a float.

    If the *first* number appears before the first "_" character,
    the *second* number in the string is returned instead.

    This allows for flexible prefixes such as "S1_", "P2_", "AaBB3_",
    etc., where the leading number in the prefix should be ignored and
    the relevant number comes later in the filename.

    Raises ValueError if no suitable number is found.
    """
    # Find the position of the first underscore (if any)
    first_underscore = fname.find("_")

    # Find all integers or decimals in the string (e.g., "123", "123.45")
    matches = list(re.finditer(r'(\d+(\.\d+)?)', fname))

    if not matches:
        raise ValueError(f"No number found in {fname!r}")

    # If there is an underscore and the first number appears before it,
    # and there is a second number, use the second one.
    if first_underscore != -1:
        first_num_start = matches[0].start()
        if first_num_start < first_underscore and len(matches) >= 2:
            return float(matches[1].group(1))

    # Otherwise, fall back to the first number
    return float(matches[0].group(1))


In [4]:
import os
import pickle
import pandas as pd

def flatten_cross_histograms(ch_dict, subject, intensity):
    """
    ch_dict: the dict under 'Cross_histograms' for one file
    subject: str, e.g. 'AB01'
    intensity: float
    returns: DataFrame with columns [Muscle_pair, MN_index, Subject, Intensity, ...flattened metrics...]
    """
    rows = []
    # Remove the 'cross histogram' key = stored for plotting purposes, not necessary here
    ch_dict.pop('cross_histograms', None)
    for muscle_pair, neuron_map in ch_dict.items():
        # extract the pre-<-> substring once
        muscle_of_MU = muscle_pair.split('<->', 1)[0]
        for mn_idx, mdata in neuron_map.items():
            row = {
                'muscle_pair':    muscle_pair,
                'muscle_of_MU':   muscle_of_MU,
                'MU_idx':       int(mn_idx),
                'subject':        subject,
                'intensity':      intensity
            }
            # now flatten the metrics under mdata
            for key, val in mdata.items():
                if isinstance(val, dict):
                    # one more level of nesting
                    for subk, subv in val.items():
                        if isinstance(subv, dict):
                            # final level: numeric entries
                            for meas, v in subv.items():
                                row[f"{key}_{subk}_{meas}"] = float(v)
                        else:
                            # numeric entry under key->subk
                            row[f"{key}_{subk}"] = float(subv)
                else:
                    # direct numeric entry
                    row[key] = float(val)
            rows.append(row)
    return pd.DataFrame(rows)

# Loop through pickle files
main_df = []
for pickle_file in os.listdir(root_path):
    if not pickle_file.lower().endswith('.pkl'):
        continue
    # ignore the "analyzes_parameters.pkl" file
    if "parameters" in pickle_file.lower():
        continue
    subject  = os.path.basename(pickle_file)[:2] # Keep the first two characters (for S1, S2...)
    intensity = extract_number(pickle_file)
    with open(os.path.join(root_path, pickle_file),'rb') as f:
        data = pickle.load(f)
    ch = data.get('Cross_histograms', {})
    # # Remove any key containing 'APB'
    # ch = {k: v for k, v in ch.items() if 'APB' not in k}
    # ch['cross_histograms'] = {k: v for k, v in ch['cross_histograms'].items() if 'APB' not in k}
    df_chunk = flatten_cross_histograms(ch, subject, intensity)
    main_df.append(df_chunk)

# combine all files
main_df = pd.concat(main_df, ignore_index=True)


In [5]:
df_mu_properties = pd.read_csv(f"{root_path}{mu_properties_filename}")
df_mu_properties['intensity'] = df_mu_properties['condition'].apply(extract_number)

In [6]:
main_df = main_df.merge(
    df_mu_properties[
      ['subject','intensity','muscle_of_MU','MU_idx',
       'condition','Recruitment_threshold_mean',
       'firing_rates_mean','cov_mean']
    ],
    left_on = ['subject','intensity','muscle_of_MU','MU_idx'],
    right_on= ['subject','intensity','muscle_of_MU','MU_idx'],
    how='left'
)


In [7]:
main_df

,muscle_pair,muscle_of_MU,MU_idx,subject,intensity,inhibited_forward_raw_area,inhibited_forward_corrected_area,inhibited_forward_z_score,inhibited_forward_p_val,inhibited_forward_null_mean,...,combined_delay_backward_IPSP,combined_hist_plateau_duration,combined_proportion_of_prob_within_plateau_duration,combined_r2_full,combined_r2_base,combined_n_spikes,condition,Recruitment_threshold_mean,firing_rates_mean,cov_mean
0,VM<->VM,VM,0,S1,10.0,-0.003369,-0.002647,-4.361873,6.447693e-06,-0.000722,...,0.039551,0.151855,0.756072,0.996133,0.970730,66677.0,Quadriceps_10percent,9.463945,10.047576,0.288856
1,VM<->VM,VM,1,S1,10.0,-0.007477,-0.004864,-4.040429,2.667674e-05,-0.002614,...,0.007324,0.135742,0.740354,0.998160,0.993264,82403.0,Quadriceps_10percent,4.586200,12.376367,0.084128
2,VM<->VM,VM,2,S1,10.0,-0.004799,-0.003921,-5.506787,1.827209e-08,-0.000878,...,0.042480,0.160156,0.788653,0.997485,0.976147,72280.0,Quadriceps_10percent,9.360852,10.027275,0.157655
3,VM<->VM,VM,3,S1,10.0,-0.007590,-0.006364,-7.099957,6.239772e-13,-0.001225,...,0.035645,0.157227,0.785207,0.994273,0.979081,72469.0,Quadriceps_10percent,9.984160,10.279303,0.124270
4,VM<->VM,VM,4,S1,10.0,-0.005885,-0.004403,-4.589524,2.221288e-06,-0.001482,...,0.009277,0.151367,0.779904,0.998010,0.985212,77722.0,Quadriceps_10percent,7.647711,10.980525,0.107836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,VL<->VL,VL,68,S1,10.0,-0.004846,-0.004778,-46.887846,0.000000e+00,-0.000068,...,0.046387,0.153320,0.724637,0.997767,0.991288,346948.0,Quadriceps_10percent,10.080313,9.362101,0.138764
142,VL<->VL,VL,69,S1,10.0,-0.005668,-0.005381,-22.143526,6.021829e-109,-0.000287,...,0.030273,0.133789,0.727419,0.998161,0.996674,470075.0,Quadriceps_10percent,3.653874,12.086696,0.074825
143,VL<->VL,VL,70,S1,10.0,-0.003181,-0.002423,-5.919517,1.614446e-09,-0.000758,...,0.033203,0.144531,0.657771,0.997652,0.994835,307785.0,Quadriceps_10percent,9.082319,8.029950,0.287274
144,VL<->VL,VL,71,S1,10.0,-0.006251,-0.005609,-13.218228,3.443370e-40,-0.000642,...,0.034180,0.153320,0.726569,0.996652,0.993791,224391.0,Quadriceps_10percent,10.027545,NaN,NaN


In [8]:
def reorganize_df(main_df):
    # 1) Map your input‐prefix → perspective
    prefix2persp = {
        'inhibited':  'other_MUs_as_ref',
        'inhibiting': 'MU_as_ref',
        'combined':   'combined',
    }
    prefixes = list(prefix2persp)

    # 2) Split out “static” vs “dynamic” columns
    dyn = [c for c in main_df.columns
           if any(c.startswith(pref + "_") for pref in prefixes)]
    stat = [c for c in main_df.columns if c not in dyn]

    rows = []
    for _, orig in main_df.iterrows():
        for pref, persp in prefix2persp.items():
            # pick just the columns for this prefix
            block = [c for c in dyn if c.startswith(pref + "_")]

            # determine which side each column belongs to:
            #   forward, backward, asymmetry, or “both” (no keyword)
            colinfo = []
            for c in block:
                key = c[len(pref)+1:]
                if "_forward_" in key:
                    side = "forward"
                    meas = key.split("_forward_",1)[1]
                elif "_backward_" in key:
                    side = "backward"
                    meas = key.split("_backward_",1)[1]
                elif "_asymmetry_" in key:
                    side = "asymmetry"
                    meas = key.split("_asymmetry_",1)[1]
                else:
                    side = "both"
                    meas = key
                colinfo.append((c, side, meas))

            # now build one “inhibited” row, one “inhibiting” row
            for direction in ("inhibited","inhibiting"):
                # decide which side we want:
                if persp in ("MU_as_ref","combined"):
                    want = "backward" if direction=="inhibited" else "forward"
                else:
                    # other_MUs_as_ref flips them
                    want = "forward"  if direction=="inhibited" else "backward"

                row = {c: orig[c] for c in stat}
                row["perspective"] = persp
                row["direction"]   = direction

                for col, side, meas in colinfo:
                    # we always include "both" measures,
                    # include forward/backward only if it matches "want",
                    # and include asymmetry same as forward/backward:
                    if side == "both" or side == want or side == "asymmetry":
                        v = orig[col]
                        # flip/invert asymmetries if needed
                        if side=="asymmetry" and direction=="inhibited":
                            # measure name ends either in "_diff" or "_ratio"
                            if meas.endswith("_asym_diff"):
                                v = -v
                            elif meas.endswith("_asym_ratio"):
                                v = np.nan if v==0 else 1.0/v
                        row[meas] = v

                rows.append(row)

    return pd.DataFrame(rows)


In [9]:
# remap the columns with backward_... and forward_... into a single set of columnn,
# splitting the backward_ and forward_ into the 'inhibited' and 'inhibiting' direction rows with the following logic:
# if perspective == 'MU_as_ref' or 'combined', then 'backward_' columns go to 'inhibited' row; 'forward_' columns go to 'inhibiting' row
# if perspective == 'other_MUs_as_ref', then 'backward_' columns go to 'inhibiting' row; 'forward_' columns go to 'inhibited' row
def remap_fwd_bwd_to_inhibiting_inhibited(df):
    # 1) which columns never change
    static = [
        'muscle_pair','muscle_of_MU','MU_idx','subject','intensity',
        'condition','Recruitment_threshold_mean',
        'firing_rates_mean','cov_mean',
        'perspective','direction'
    ]
    # 2) the “paired” measures that currently sit in forward_*/backward_* columns
    paired = [
        'raw_area','corrected_area',
        'z_score','p_val','null_mean','null_std'
    ]
    # 3) the “always‐copy” measures
    copy_through = [
        'sync_height','sync_time','IPSP',
        'hist_plateau_duration',
        'proportion_of_prob_within_plateau_duration',
        'r2_full','r2_base','n_spikes'
    ]
    # 4) and the two asymmetry outputs (we’ll recompute their sign/reciprocal too)
    #    your exploded frame has:
    #      asymmetry_raw_area_asym_ratio
    #      asymmetry_raw_area_asym_diff
    #      asymmetry_corrected_area_asym_ratio
    #      asymmetry_corrected_area_asym_diff
    #    we’ll read those and write out clean:
    asym_cols = [
        ('asymmetry_raw_area_asym_diff','asymmetry_diff_raw'),
        ('asymmetry_corrected_area_asym_diff','asymmetry_diff_corrected'),
        ('asymmetry_raw_area_asym_ratio','asymmetry_ratio_raw'),
        ('asymmetry_corrected_area_asym_ratio','asymmetry_ratio_corrected'),
    ]

    rows = []
    for _, row in df.iterrows():
        base_p = row['perspective']
        dir_   = row['direction']
        # MU_as_ref & combined: forward→inhibiting, backward→inhibited
        # other_MUs_as_ref:    forward→inhibited,  backward→inhibiting
        if base_p in ('MU_as_ref','combined'):
            fwd_for = 'inhibiting'
            bwd_for = 'inhibited'
        else:
            fwd_for = 'inhibited'
            bwd_for = 'inhibiting'

        r = {c: row[c] for c in static}
        # 1) remap the paired forward_/backward_ columns:
        for m in paired:
            fwd_val = row[f'forward_{m}']
            bwd_val = row[f'backward_{m}']
            # if this row’s direction matches the “forward” mapping, pick fwd_val, else bwd_val
            r[m] = fwd_val if dir_ == fwd_for else bwd_val

        # 2) copy through the “always” elements
        for m in copy_through:
            r[m] = row[m]

        # 3) asymmetries: pick the raw/corrected inputs, then flip/invert on inhibited
        for src_col, tgt_col in asym_cols:
            v = row[src_col]
            if src_col.endswith('_diff'):
                if (base_p == 'other_MUs_as_ref') and (dir_ == 'inhibited'):
                    r[tgt_col] = v # assymetry diff = forward - backward = inhibited - inhibiting
                elif (base_p == 'other_MUs_as_ref') and (dir_ == 'inhibiting'):
                    r[tgt_col] = -v # assymetry diff = backward - forward = inhibiting - inhibited
                elif (base_p == 'MU_as_ref') and (dir_ == 'inhibited'):
                    r[tgt_col] = -v # assymetry diff = backward - forward = inhibited - inhibiting
                elif (base_p == 'MU_as_ref') and (dir_ == 'inhibiting'):
                    r[tgt_col] = v # assymetry diff = forward - backward = inhibiting - inhibited
            else:  # ratio
                if (base_p == 'other_MUs_as_ref') and (dir_ == 'inhibited'):
                    r[tgt_col] = v # assymetry ratio = forward / backward = inhibited / inhibiting
                elif (base_p == 'other_MUs_as_ref') and (dir_ == 'inhibiting'):
                    r[tgt_col] = np.nan if v==0 else 1.0/v # assymetry ratio = backward / forward = inhibiting / inhibited
                elif (base_p == 'MU_as_ref') and (dir_ == 'inhibited'):
                    r[tgt_col] = np.nan if v==0 else 1.0/v # assymetry ratio = backward / forward = inhibited / inhibiting
                elif (base_p == 'MU_as_ref') and (dir_ == 'inhibiting'):
                    r[tgt_col] = v # assymetry ratio = forward / backward = inhibiting / inhibited

        rows.append(r)

    return pd.DataFrame(rows)


In [10]:
main_df = reorganize_df(main_df)
main_df = remap_fwd_bwd_to_inhibiting_inhibited(main_df)

In [11]:
# With the re-organizing scheme above, the delay/timing (time of lowest autocorr) gets renamed simply "IPSP"
# Changing the column name so that it becomes more explicit
main_df = main_df.rename(columns={"IPSP": "IPSP_timing_of_trough"})

In [12]:
# FILTERING DATA FRAME
# Remove APB
def exclude_muscles(df, bad_muscle_list):
    bad = set(bad_muscle_list)
    return df.loc[
        ~( df['muscle_of_MU'].isin(bad) | 
           df['muscle_pair'].apply(lambda s: any(m in s for m in bad)) )
    ].reset_index(drop=True)
filtered_df = exclude_muscles(main_df, ['APB'])
# # Choose perspective
# filtered_df = filtered_df[filtered_df['perspective']=='MU_as_ref']
# # Only 10% intensity
# filtered_df = filtered_df[filtered_df['intensity']==10]
# Save as a csv
# filtered_df.to_csv(f"{root_path}experimental_data_dataframe.csv")
# filtered_df

In [13]:
filtered_df = filtered_df[filtered_df['perspective'] != 'combined']

nb_spikes_per_perspective = (
    filtered_df.groupby(['muscle_pair','intensity','subject','perspective','direction'], as_index=False)
      .agg(n_spikes=('n_spikes','mean'))
)
nb_spikes_per_perspective['Quadriceps_interaction'] = nb_spikes_per_perspective['muscle_pair'].isin(['VM<->VL','VL<->VM'])
nb_spikes_per_perspective['TricepsSurae_interaction'] = nb_spikes_per_perspective['muscle_pair'].isin(['SOL<->GM','GM<->SOL'])


In [14]:
# find which one of the two 'muscle_pair' in 'Quadriceps_interaction' and 'TricepsSurae_interaction'has the highest n_spikes, across subjects an intensities
# Output it in a single list (for example ['VM<->VL' and 'SOL<->GM'])
# Then add a column "pair_to_select" to filtered_df. It is true for every muscle_pair (strings constructed like so "A<->B") for which A=B, or for which the string is in the list we just constructed
# Then, finally, please add a column "full_spellout", which, for each row, takes muscle A and B from muscle_pair and writes f"A_inhibits_B" if direction=="inhibiting" and f"A_inhibited_by_B" if direction=="inhibited"

# 1) find top‐spiking pair in each interaction group
# ---------------------------------------------------
# mean over subjects & intensities, for just the quad rows:
quad = nb_spikes_per_perspective[ nb_spikes_per_perspective['Quadriceps_interaction'] ]
if quad.size > 1:
    top_quad = quad.groupby('muscle_pair')['n_spikes'].mean().idxmax()
else:
    top_quad = "None"

# same for triceps surae
tri = nb_spikes_per_perspective[ nb_spikes_per_perspective['TricepsSurae_interaction'] ]
if tri.size > 1:
    top_tri  = tri.groupby('muscle_pair')['n_spikes'].mean().idxmax()
else:
    top_tri = "None"

selected_pairs = { top_quad, top_tri }


# 2) tag filtered_df rows to keep
# --------------------------------
# split muscle_pair → A, B
AB = filtered_df['muscle_pair'].str.split('<->', expand=True)
filtered_df['muscle_A'] = AB[0]
filtered_df['muscle_B'] = AB[1]

# keep if self-pair (A==B) or in our winners
filtered_df['pair_to_select'] = (
    (filtered_df['muscle_A'] == filtered_df['muscle_B'])
  | (filtered_df['muscle_pair'].isin(selected_pairs))
)

# ———————————————————————————————————————————————————————————
# 3) Re-introduce the “missing” cross-pairs by mirroring the selected ones
# ———————————————————————————————————————————————————————————
filtered_df = filtered_df[filtered_df['pair_to_select']]
to_add = []
# look only at the kept CROSS-pairs (A!=B) that are in selected_pairs
mask = (filtered_df['muscle_A'] != filtered_df['muscle_B']) & \
       (filtered_df['muscle_pair'].isin(selected_pairs))

for _, row in filtered_df[mask].iterrows():
    A, B = row['muscle_A'], row['muscle_B']
    new = row.copy()

    # swap the identities
    new['muscle_of_MU'] = B # doesn't correspond to an actual MU anymore (just mmirroring the result of the MU of the paired muscle)
    new['MU_idx'] = np.nan
    new['muscle_A'], new['muscle_B'] = B, A
    new['muscle_pair'] = f"{B}<->{A}"

    # invert direction
    new['direction'] = (
        'inhibited'   if row['direction']=='inhibiting'
        else 'inhibiting'
    )

    # swap perspective MU_as_ref <-> other_MUs_as_ref, leave combined alone
    if row['perspective']=='MU_as_ref':
        new['perspective'] = 'other_MUs_as_ref'
    elif row['perspective']=='other_MUs_as_ref':
        new['perspective'] = 'MU_as_ref'
    # if it's 'combined' we just leave it

    to_add.append(new)

# append and reset_index
if to_add:
    filtered_df = pd.concat(
        [filtered_df, pd.DataFrame(to_add)],
        ignore_index=True
    )


# 4) full_spellout column (mostly to not get confused)
# -----------------------
def _spellout_pair(row):
    A = row['muscle_A']
    B = row['muscle_B']
    if row['direction'] == 'inhibiting':
        return [f"{A}_inhibits_{B}", f"{B}_inhibited_by_{A}"]
    else:  # 'inhibited'
        return [f"{A}_inhibited_by_{B}", f"{B}_inhibits_{A}"]

filtered_df['full_spellout'] = filtered_df.apply(_spellout_pair, axis=1)

# 5) Compute mean n_spikes per perspective across everything
# -----------------------
mean_spikes = (
    nb_spikes_per_perspective
    .groupby('perspective', as_index=False)['n_spikes']
    .mean()
)
# Find which perspective has the highest mean
best_persp = mean_spikes.loc[mean_spikes['n_spikes'].idxmax(), 'perspective']

# Just some dirty code to quickly switch the perspective and direction
# best_persp = "other_MUs_as_ref" # "MU_as_ref"
# direction_to_use = "inhibiting"

# Add a boolean “use_this_perspective” column to filtered_df
filtered_df['use_this_perspective'] = filtered_df['perspective'] == best_persp

print(f"▶︎ Using perspective “{best_persp}” for all downstream analyses.")

direction_to_use = "inhibited" # since we use the "other_MUs_as_ref" perspective (to maximize spike number), the "inhibited" direction is the one looking in the "forward" direction in the histogram
filtered_df['use_this_direction'] = filtered_df['direction']==direction_to_use
print(f"▶︎ Using direction 'inhibited' for all downstream analyses.\nsince we use the 'other_MUs_as_ref' perspective (to maximize spike number), the 'inhibited' direction is the one looking in the 'forward' direction in the histogram")

# Save as a csv
filtered_df.to_csv(f"{root_path}experimental_data_dataframe_RAW.csv")
filtered_df



▶︎ Using perspective “other_MUs_as_ref” for all downstream analyses.
▶︎ Using direction 'inhibited' for all downstream analyses.
since we use the 'other_MUs_as_ref' perspective (to maximize spike number), the 'inhibited' direction is the one looking in the 'forward' direction in the histogram


,muscle_pair,muscle_of_MU,MU_idx,subject,intensity,condition,Recruitment_threshold_mean,firing_rates_mean,cov_mean,perspective,...,asymmetry_diff_raw,asymmetry_diff_corrected,asymmetry_ratio_raw,asymmetry_ratio_corrected,muscle_A,muscle_B,pair_to_select,full_spellout,use_this_perspective,use_this_direction
0,VM<->VM,VM,0.0,S1,10.0,Quadriceps_10percent,9.463945,10.047576,0.288856,other_MUs_as_ref,...,0.003450,0.003204,0.494028,0.452414,VM,VM,True,"[VM_inhibited_by_VM, VM_inhibits_VM]",True,True
1,VM<->VM,VM,0.0,S1,10.0,Quadriceps_10percent,9.463945,10.047576,0.288856,other_MUs_as_ref,...,-0.003450,-0.003204,2.024177,2.210363,VM,VM,True,"[VM_inhibits_VM, VM_inhibited_by_VM]",True,False
2,VM<->VM,VM,0.0,S1,10.0,Quadriceps_10percent,9.463945,10.047576,0.288856,MU_as_ref,...,0.002783,0.002585,0.603398,0.572910,VM,VM,True,"[VM_inhibited_by_VM, VM_inhibits_VM]",False,True
3,VM<->VM,VM,0.0,S1,10.0,Quadriceps_10percent,9.463945,10.047576,0.288856,MU_as_ref,...,-0.002783,-0.002585,1.657281,1.745475,VM,VM,True,"[VM_inhibits_VM, VM_inhibited_by_VM]",False,False
4,VM<->VM,VM,1.0,S1,10.0,Quadriceps_10percent,4.586200,12.376367,0.084128,other_MUs_as_ref,...,-0.003074,-0.003268,1.698201,3.048755,VM,VM,True,"[VM_inhibited_by_VM, VM_inhibits_VM]",True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,VL<->VM,VL,NaN,S1,10.0,Quadriceps_10percent,9.979341,NaN,NaN,other_MUs_as_ref,...,-0.002173,-0.002278,2.339432,45.847826,VL,VM,True,"[VL_inhibited_by_VM, VM_inhibits_VL]",True,True
384,VL<->VM,VL,NaN,S1,10.0,Quadriceps_10percent,9.987426,9.439659,0.181869,MU_as_ref,...,0.000970,0.000943,0.821529,0.800869,VL,VM,True,"[VL_inhibits_VM, VM_inhibited_by_VL]",False,False
385,VL<->VM,VL,NaN,S1,10.0,Quadriceps_10percent,9.987426,9.439659,0.181869,MU_as_ref,...,-0.000970,-0.000943,1.217242,1.248644,VL,VM,True,"[VL_inhibited_by_VM, VM_inhibits_VL]",False,True
386,VL<->VM,VL,NaN,S1,10.0,Quadriceps_10percent,9.987426,9.439659,0.181869,other_MUs_as_ref,...,0.001926,0.001933,0.595733,0.537238,VL,VM,True,"[VL_inhibits_VM, VM_inhibited_by_VL]",True,False


In [15]:
filtered_df_to_save = filtered_df.copy()

# Get for each [subject, intensity and muscle] the summary stats (min, max, mean, std) of MU properties values:
# Recruitment_threshold_mean, firing_rates_mean, cov_mean
MU_properties_colnames = ["Recruitment_threshold_mean", "firing_rates_mean", "cov_mean"]
MU_properties_summary_stats = {}
mask_within = filtered_df_to_save['muscle_A']==filtered_df_to_save['muscle_B']
for subjecti in filtered_df_to_save['subject'].unique():
    MU_properties_summary_stats[subjecti] = {}
    mask_subject = filtered_df_to_save['subject']==subjecti
    for intensity_i in filtered_df_to_save['intensity'].unique():
        MU_properties_summary_stats[subjecti][intensity_i] = {}
        mask_intensity = filtered_df_to_save['intensity']==intensity_i
        for muscle_i in filtered_df_to_save['muscle_A'].unique():
            MU_properties_summary_stats[subjecti][intensity_i][muscle_i] = {}
            mask_muscle = filtered_df_to_save['muscle_of_MU']==muscle_i
            mask_combined = (mask_within & mask_subject & mask_intensity & mask_muscle)
            for mu_property_i in MU_properties_colnames:
                series = filtered_df_to_save.loc[mask_combined, mu_property_i]
                MU_properties_summary_stats[subjecti][intensity_i][muscle_i][mu_property_i] = {
                    "mean": series.mean(),
                    "std": series.std(),
                    "min": series.min(),
                    "max": series.max()}

# Filter chosen perspective and direction
filtered_df_to_save = filtered_df_to_save[filtered_df_to_save["use_this_perspective"]]
print(f"Chosen perspective: {best_persp}")

def swap_spellout(lst):
    # lst is e.g. ["A_inhibits_B","B_inhibited_by_A"]
    return [lst[1], lst[0]]
mask_between_muscles = filtered_df_to_save['muscle_A']!=filtered_df_to_save['muscle_B']
mask_other_dir = filtered_df_to_save["direction"] != direction_to_use
mask = mask_between_muscles & mask_other_dir
pairs = filtered_df_to_save.loc[mask, "muscle_pair"].str.split("<->", expand=True)
filtered_df_to_save.loc[mask, "muscle_pair"] = pairs[1] + "<->" + pairs[0]
filtered_df_to_save.loc[mask,'muscle_of_MU'] = pairs[1]
filtered_df_to_save.loc[mask, 'full_spellout'] = (filtered_df_to_save # Not needed, I got confused at some point and added that
      .loc[mask, 'full_spellout']
      .apply(swap_spellout))
# already switched
# filtered_df_to_save.loc[mask,'muscle_A'] = pairs[1]
# filtered_df_to_save.loc[mask,'muscle_B'] = pairs[0]
filtered_df_to_save.loc[mask,'direction'] = direction_to_use
for subjecti in filtered_df_to_save.loc[mask,'subject'].unique(): # ['DeFr']: for debugging
    mask_subject = filtered_df_to_save['subject']==subjecti
    for intensity_i in filtered_df_to_save.loc[mask,'intensity'].unique():
        mask_intensity = filtered_df_to_save['intensity']==intensity_i
        for muscle_i in filtered_df_to_save.loc[mask,'muscle_of_MU'].unique(): # ['VL','GM']: for debugging
            mask_muscle = filtered_df_to_save.loc[mask,'muscle_of_MU']==muscle_i
            mask_combined = (mask & mask_subject & mask_intensity & mask_muscle)
            for mu_property_i in MU_properties_colnames:
                series = filtered_df_to_save[mask_combined].copy()
                filtered_df_to_save.loc[mask_combined,mu_property_i] = np.random.normal(
                    MU_properties_summary_stats[subjecti][intensity_i][muscle_i][mu_property_i]['mean'],
                    MU_properties_summary_stats[subjecti][intensity_i][muscle_i][mu_property_i]['std'],
                    series.shape[0])
    
filtered_df_to_save = filtered_df_to_save[filtered_df_to_save["use_this_direction"]]
print(f"Chosen direction: {direction_to_use}")
# Save as a csv
filtered_df_to_save.to_csv(f"{root_path}experimental_data_dataframe_reorganized_and_filtered_dir_{direction_to_use}_persp_{best_persp}.csv") # reorganized version, to be used

# Filter min values
# filtered_df_to_save = filtered_df_to_save[filtered_df_to_save['n_spikes']>5000]
# filtered_df_to_save = filtered_df_to_save[filtered_df_to_save['r2_base']>0.1]
# filtered_df_to_save = filtered_df_to_save[filtered_df_to_save['r2_full']>0.75]

Chosen perspective: other_MUs_as_ref
Chosen direction: inhibited


In [16]:
filtered_df_to_save

,muscle_pair,muscle_of_MU,MU_idx,subject,intensity,condition,Recruitment_threshold_mean,firing_rates_mean,cov_mean,perspective,...,asymmetry_diff_raw,asymmetry_diff_corrected,asymmetry_ratio_raw,asymmetry_ratio_corrected,muscle_A,muscle_B,pair_to_select,full_spellout,use_this_perspective,use_this_direction
0,VM<->VM,VM,0.0,S1,10.0,Quadriceps_10percent,9.463945,10.047576,0.288856,other_MUs_as_ref,...,0.003450,0.003204,0.494028,0.452414,VM,VM,True,"[VM_inhibited_by_VM, VM_inhibits_VM]",True,True
4,VM<->VM,VM,1.0,S1,10.0,Quadriceps_10percent,4.586200,12.376367,0.084128,other_MUs_as_ref,...,-0.003074,-0.003268,1.698201,3.048755,VM,VM,True,"[VM_inhibited_by_VM, VM_inhibits_VM]",True,True
8,VM<->VM,VM,2.0,S1,10.0,Quadriceps_10percent,9.360852,10.027275,0.157655,other_MUs_as_ref,...,-0.000775,-0.000725,1.192554,1.226921,VM,VM,True,"[VM_inhibited_by_VM, VM_inhibits_VM]",True,True
12,VM<->VM,VM,3.0,S1,10.0,Quadriceps_10percent,9.984160,10.279303,0.124270,other_MUs_as_ref,...,-0.004268,-0.004135,2.284836,2.854376,VM,VM,True,"[VM_inhibited_by_VM, VM_inhibits_VM]",True,True
16,VM<->VM,VM,4.0,S1,10.0,Quadriceps_10percent,7.647711,10.980525,0.107836,other_MUs_as_ref,...,-0.001232,-0.000910,1.264867,1.260567,VM,VM,True,"[VM_inhibited_by_VM, VM_inhibits_VM]",True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,VL<->VM,VL,NaN,S1,10.0,Quadriceps_10percent,9.206214,8.456785,0.163556,other_MUs_as_ref,...,-0.000113,-0.000255,1.071898,1.285144,VL,VM,True,"[VL_inhibited_by_VM, VM_inhibits_VL]",True,True
375,VL<->VM,VL,NaN,S1,10.0,Quadriceps_10percent,6.449916,9.158443,0.085265,other_MUs_as_ref,...,0.000765,0.000902,0.690542,0.410558,VL,VM,True,"[VL_inhibited_by_VM, VM_inhibits_VL]",True,True
379,VL<->VM,VL,NaN,S1,10.0,Quadriceps_10percent,9.861770,10.107177,0.093927,other_MUs_as_ref,...,0.001906,0.001861,0.594529,0.550823,VL,VM,True,"[VL_inhibited_by_VM, VM_inhibits_VL]",True,True
383,VL<->VM,VL,NaN,S1,10.0,Quadriceps_10percent,9.979341,NaN,NaN,other_MUs_as_ref,...,-0.002173,-0.002278,2.339432,45.847826,VL,VM,True,"[VL_inhibited_by_VM, VM_inhibits_VL]",True,True
